In [1]:
import requests
import json
from IPython.display import HTML, display

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '👎',
          'THUMBS_UP': '👍',
          'LAUGH': '😄'}

In [4]:
issues = [126, 122, 189, 153, 25, 158, 205, 270, 291,
      306, 96, 76, 118, 174, 311, 286, 199, 31, 115,
      64, 132, 46, 27, 106, 183, 50, 98, 190, 93,
      243, 266, 295, 231, 240, 244, 175, 232, 238,
      289, 230, 309, 308, 317, 218, 234, 298, 297,
      229, 233, 302, 265, 237, 59, 176, 301, 149, 58,
      134, 382, 217, 299, 239, 363, 360, 210, 359,
      379, 381, 34, 332, 293, 28, 378, 377, 383]

In [5]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

In [29]:
def tabulate_emojis(query_result, issue_num):
    data = [
        ["Person"],
        ["Rating"]]
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        for edge in reaction['users']['edges']:
            if edge['node']['name']:
                data[0].append(edge['node']['name'])
            else:
                data[0].append(edge['node']['login'])
        
            data[1].append(emojis[reaction['content']])           

    print(str(issue_num) + ": " + query_result['data']['repository']['issue']['title'])
    display(HTML(
   '<table><tr>{}</tr></table>'.format(
       '</tr><tr>'.join(
           '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
       )
    ))


In [33]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

**TODO**
1. Have queries that work for the other issues not currently working for - WORKS FOR ALL, JUST HAD TO FILTER THOSE WITH NO REACTIONS/EMOJIS
2. Save the real names preferably rather than github id - DONE
3. Incorporate the issues list from github directly
4. Think about how to filter by availability, using forecast data for people and project
2. Think of best ways to save the output, this simple table? csv? Eventually will go in main Wimbledon planner

In [34]:
for issue_num in issues:
    modified_query = query.replace("X", str(issue_num))
    result = run_query(modified_query)  # Execute the query
    if query_result_contains_emojis(result):
        tabulate_emojis(result, issue_num)

205: Crypto-Friendly Models for Faster Encrypted Prediction - Adria Gascon


Person,cathiest
Rating,👍


270: The Turing Way - reproducible research practice guide - Phase 1


Person,Giovanni Colavizza,Camila Rangel Smith,Radka Jersakova,Sarah Gibson,myyong,Oliver Strickson,Tomas Lazauskas,nbarlowATI,Tim Hobson,miguelmorin,James Robinson,Kasra Hosseini,Louise Bowler,Evelina Gabasova
Rating,👍,👍,👍,👍,👎,👎,👎,👎,👎,👎,👎,👎,😄,😄


291: Biobank algorithm - Classify Fitbit data with parallel TensorFlow 


...

Person,Sarah Gibson,James Robinson,Louise Bowler,Jim Madge,Eric Daub,Jack Roberts,nbarlowATI,Evelina Gabasova,Oscar T Giles,Radka Jersakova,Roly Perera,myyong,AshwiniKV,Kasra Hosseini,miguelmorin,Tim Hobson,Ed Chalstrey
Rating,👍,👍,👍,👍,👍,👍,👍,👍,👍,👍,👎,👎,👎,👎,👎,👎,😄


332: Robust, Scalable Sequential Monte Carlo (Adam Johansen & Theo Damoulas)


Person,Louise Bowler,Jim Madge,Eric Daub,Ed Chalstrey,James Geddes,Oscar T Giles,miguelmorin,Tomas Lazauskas,Tim Hobson,myyong,Sarah Gibson,Roly Perera,Camila Rangel Smith,Evelina Gabasova,Kasra Hosseini,AshwiniKV,Radka Jersakova,James Robinson,nbarlowATI
Rating,👍,👍,👍,👍,👍,👍,👍,👍,👍,👎,👎,👎,👎,👎,👎,👎,👎,😄,😄


293: Wrattler 2019-20 - Tomas Petricek


Person,Sarah Gibson
Rating,😄
